In [ ]:
import tqdm
import torchvision
import torch
import torch.nn as nn
import os
import time
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import torchvision.models as models
from torchsummary import summary

writer = SummaryWriter()

train_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
target_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


def make_dataset(dir):
    items = []
    labels = []

    for rootdir, dirs, files in tqdm(os.walk(dir)):
        #         print(rootdir)
        for subdir in dirs:
            #print(subdir)
            for files1 in os.listdir(os.path.join(rootdir, subdir)):
                image_path = os.path.join(os.path.join(rootdir, subdir), files1)
                items.append(image_path)
                labels.append(subdir)
    print(len(items))
    return items, labels

def create_dataset(mode, dir, random_transform=None, target_transform=None):
    images, labels = make_dataset(dir)
    #print(images)
    lenth = len(images)
    labels = len(labels)
    print(lenth)
    print(labels)
    return images, labels


def load_dataset(trainBatchsize, valBatchSize):
    train_img_dir = r"D:\Animal_Classification\dataset\Animal\train"
    val_img_dir = r"D:\Animal_Classification\dataset\Animal\val"

#     training_Set = create_dataset('train', train_img_dir, random_transform=train_transform,
#                                   target_transform=target_transform)
#     validation_Set = create_dataset('val', val_img_dir, random_transform=train_transform,
#                                     target_transform=target_transform)

    training_Set = torchvision.datasets.ImageFolder(train_img_dir , transform=train_transform)
    validation_Set = torchvision.datasets.ImageFolder(val_img_dir , transform=train_transform)
    
    trainloader = torch.utils.data.DataLoader(training_Set, trainBatchsize, shuffle=True)
    valloader = torch.utils.data.DataLoader(training_Set, trainBatchsize, shuffle=False)
    return trainloader, valloader

def Model(inputsize):
    model = models.mobilenet_v2(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs,5)
    return model


def save_checkpoint(state, filename):
    torch.save(state, filename)


def train(model, train_loader, criterion, optimizer, epoch, step=50):
    model.train()
    
    for i, (images, labels) in enumerate(train_loader):
        #print(torch.cuda.is_available())
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        output = model(images)
        loss = criterion(output,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%step==0:
            print('EPOCH {} | ITER {} | AVG_LOSS {}'.format(epoch, i, loss))
        writer.add_scalar('TRAIN_LOSS', loss, epoch)
        
    return loss


def main():
    model = Model((3,224,224))
    print(torch.cuda.is_available())
    if torch.cuda.is_available():
        model = model.cuda()
        print("using gpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    epochs = 30
    train_batch_size = 1;
    val_batch_size = 1;

    train_loader, val_loader = load_dataset(train_batch_size, val_batch_size)
    
    #     history = open('./checkpoint/history.csv','w')
    #     history.write('epochs,trainloss,valloss\n')

    for epoch in range(0,epochs):
        print(epoch)
        start = time.time()
        train_loss = train(model, train_loader, criterion, optimizer, epoch, step=30)
        val_loss = train(model, val_loader, criterion, optimizer, epoch)
        print()
        print('-' * 50)
        print('EPOCH {} | LOSS {} | TIME {}'.format(epoch, train_loss, time.time() - start))
        print('-' * 50)
        print()


#         history.write('{},{},{}\n'.format(epoch, train_loss, validation_loss))
#         save_checkpoint({'epoch': epoch,'state_dict': model.state_dict(),
#             'optimizer' : optimizer.state_dict(),'loss' : train_loss}, './checkpoints/checkpoint_{}.ckpt'.format(epoch))
#     history.close()


# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    main()

# See PyCharm help at https://www.jetbrains.com/help/pycharm/


False
0
EPOCH 0 | ITER 0 | AVG_LOSS 1.434604525566101
EPOCH 0 | ITER 30 | AVG_LOSS 1.6141493320465088
EPOCH 0 | ITER 60 | AVG_LOSS 1.6288108825683594
EPOCH 0 | ITER 90 | AVG_LOSS 1.9926544427871704
EPOCH 0 | ITER 120 | AVG_LOSS 1.6816366910934448
EPOCH 0 | ITER 150 | AVG_LOSS 1.2351018190383911
EPOCH 0 | ITER 180 | AVG_LOSS 1.6761395931243896
EPOCH 0 | ITER 210 | AVG_LOSS 1.8917304277420044
EPOCH 0 | ITER 240 | AVG_LOSS 1.5717209577560425


In [21]:
nvidia-smi

NameError: name 'nvidia' is not defined